# Feature Extraction & Classification



In [1]:
from typing import Dict, Tuple, List, Union

get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [6]:
pip install spacy

In [1]:
from os.path import exists, join
import pandas as pd
import requests
#import en_core_web_md
#import spacy
from nltk.corpus import wordnet as wn



### load Data

In [2]:
# Load Datasets
def loadDataset(file_name):
    df = pd.read_csv(file_name)
    return df

df = loadDataset("Dataset/TestData/test_dataset.csv")

In [15]:
df

,ID_packet,IP_src,UDP_sport,IP_dst,UDP_dport,DNS_id,Domaine,Country,tld,DNS_qr,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,53,10.0.2.15,47533,47210,ecnavi.jp,unkown,jp,1,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,53,10.0.2.15,43722,7987,ecnavi.jp,unkown,jp,1,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,53,10.0.2.15,47296,30024,hubpages.com,unkown,com,1,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,53,10.0.2.15,47296,54860,hubpages.com,unkown,com,1,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,53,10.0.2.15,55060,48426,discover.hubpages.com,unkown,com,1,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,53,10.0.2.15,34276,43676,c1781279.ferozo.com,unkown,com,1,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,53,10.0.2.15,33341,34713,c1781279.ferozo.com,unkown,com,1,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,53,10.0.2.15,60738,44402,rusnod.ru,unkown,ru,1,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,53,10.0.2.15,50988,30583,rusnod.ru,unkown,ru,1,...,-1,-1,1,9,0,0,0,0,0,0


In [16]:
df.shape

(2126, 24)

## Linguistic Features 

In [9]:
pip install sklearn.feature_extraction.stop_words

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sklearn.feature_extraction.stop_words (from versions: none)
ERROR: No matching distribution found for sklearn.feature_extraction.stop_words


In [6]:
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as sklearn_stopwords
from nltk import ngrams as NLTK_ngrams

When we perform machine learning on text data, there are a number of different features that may prove valuable in learning how to distinguish between our classes.

Here, we break them down into "Syntactic Features" and "Semantic Features":

### Syntactic Features: 
Features that reflect the structure of our sentences.


1. **Word ngrams**: These are the length-n word sequences that occur in our data. Often, we add "BOS" and "EOS" tokens to mark the beginning and end of the sentence, respectively. For example, the "bigrams" (n=2) for the sentence "This is an example" would be {"BOS this", "this is", "is an", "an example", "example EOS"}. The order of `n` can be anything, though typically high orders of `n` result in such sparse features that they are no longer very useful. We will experiment with a few different values and ranges of `n`, and ultimately settle on "unigrams" (single words), and "bigrams" (pairs of adjacent words).
  <u>Stopwords and Word ngrams</u>: Often, high-frequency words that serve a mostly functional purpose, eg. "of", "the", "from", are found to be of little help during text classification problems, and are thus removed from the text before ngram features are extracted. We can experiment with different sets of stopwords in our ngram-generation function.


Great, now lets split into words into a big list, splitting on anything non-alphanumeric [A-Za-z0-9] (as well as punctuation) and forcing everything lowercase

In [17]:
#removing duplicates
df = df.drop_duplicates()

In [18]:
df.shape

(2126, 24)

In [33]:
#Adding columns to the dataframe df
df['1gram']=""
df['2gram']=""
df['3gram']=""
df['entropy']=""
df = df[['ID_packet', 'IP_src', 'UDP_sport', 'IP_dst', 'UDP_dport', 'DNS_id','Domaine','1gram','2gram','3gram','entropy','Country','tld', 'DNS_qr', 'DNS_qd', 'DNS_qdtype', 'type_table', 'Creation_Date_Time', 'Exp_Date_Time', 'whois_updatedDate', 'dot_count','len', 'digit_count', 'special_count', 'hyphen_count', 'double_slash','single_slash', 'at_the_rate']]

df    

,ID_packet,IP_src,UDP_sport,IP_dst,UDP_dport,DNS_id,Domaine,1gram,2gram,3gram,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,53,10.0.2.15,47533,47210,ecnavi.jp,,,,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,53,10.0.2.15,43722,7987,ecnavi.jp,,,,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,53,10.0.2.15,47296,30024,hubpages.com,,,,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,53,10.0.2.15,47296,54860,hubpages.com,,,,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,53,10.0.2.15,55060,48426,discover.hubpages.com,,,,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,53,10.0.2.15,34276,43676,c1781279.ferozo.com,,,,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,53,10.0.2.15,33341,34713,c1781279.ferozo.com,,,,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,53,10.0.2.15,60738,44402,rusnod.ru,,,,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,53,10.0.2.15,50988,30583,rusnod.ru,,,,...,-1,-1,1,9,0,0,0,0,0,0


In [23]:
words = df['Domaine']
#words = filter(None, words) # Remove empty strings


In [24]:
words

0                   ecnavi.jp
1                   ecnavi.jp
2                hubpages.com
3                hubpages.com
4       discover.hubpages.com
                ...          
2121      c1781279.ferozo.com
2122      c1781279.ferozo.com
2123                rusnod.ru
2124                rusnod.ru
2125                rusnod.ru
Name: Domaine, Length: 2126, dtype: object

# Domaine name  1-gram

From this we can now generate N-grams, lets start with a 1-gram, basically the set of all the words

*Note* : One could use a dictionary instead of a set and keeping count of the occurances gives word frequency

In [28]:
pip install sets

  Created wheel for sets: filename=sets-0.3.2-py3-none-any.whl size=14335 sha256=8ed9e7ec18c48bdef269aea4e34cf1abecc9a54a506a2f60efdc7b268a24edc0
  Stored in directory: c:\users\pc1\appdata\local\pip\cache\wheels\26\6b\c5\bc48cae07441d54bc52011a9550d775a38e7dc353c1506bfcf
  Created wheel for definitions: filename=definitions-0.2.0-py3-none-any.whl size=4573 sha256=bf70d483332514f9ddefc2a82d2763fead07558683dcef9bd31f60678b0c9f5e
  Stored in directory: c:\users\pc1\appdata\local\pip\cache\wheels\1b\24\04\358c657ef68b53b4c59d390670ab780f159515b48b16b31f72
Successfully built sets definitions
Note: you may need to restart the kernel to use updated packages.


In [34]:
import sets
for x in df.index:
    try:
        u = df.Domaine[x]
# Create set of all unique words, this throws away any information about frequency however
        gram1 = set(u)
        df['1gram'][x] = gram1
    except:
        pass

df

C:\Users\PC1\AppData\Local\Temp/ipykernel_17724/2224606195.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['1gram'][x] = gram1


,ID_packet,IP_src,UDP_sport,IP_dst,UDP_dport,DNS_id,Domaine,1gram,2gram,3gram,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,53,10.0.2.15,47533,47210,ecnavi.jp,"{a, n, ., e, j, p, i, c, v}",,,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,53,10.0.2.15,43722,7987,ecnavi.jp,"{a, n, ., e, j, p, i, c, v}",,,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,53,10.0.2.15,47296,30024,hubpages.com,"{a, g, ., o, e, p, h, s, b, m, u, c}",,,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,53,10.0.2.15,47296,54860,hubpages.com,"{a, g, ., o, e, p, h, s, b, m, u, c}",,,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,53,10.0.2.15,55060,48426,discover.hubpages.com,"{a, d, g, ., o, e, u, h, p, s, i, b, r, m, c, v}",,,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,53,10.0.2.15,34276,43676,c1781279.ferozo.com,"{z, 8, 1, ., o, e, 7, 9, m, f, 2, c, r}",,,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,53,10.0.2.15,33341,34713,c1781279.ferozo.com,"{z, 8, 1, ., o, e, 7, 9, m, f, 2, c, r}",,,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,53,10.0.2.15,60738,44402,rusnod.ru,"{d, n, ., o, s, u, r}",,,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,53,10.0.2.15,50988,30583,rusnod.ru,"{d, n, ., o, s, u, r}",,,...,-1,-1,1,9,0,0,0,0,0,0


# Domaine name  2-gram

Lets try and get the 2-gram now, which is pairs of two char from domain name. 

In [8]:
from nltk import ngrams
import sets
for x in df.index:
    try:
        gram2 =["".join(k1) for k1 in list(ngrams(df.Domaine[x],n=2))]
        df['2gram'][x] = gram2
    except:
        pass
    
df

,ID_packet,IP_src,UDP_sport,IP_dst,UDP_dport,DNS_id,Domaine,1gram,2gram,3gram,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,53,10.0.2.15,47533,47210,ecnavi.jp,"{'a', 'n', '.', 'e', 'j', 'p', 'i', 'c', 'v'}","[ec, cn, na, av, vi, i., .j, jp]",NaN,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,53,10.0.2.15,43722,7987,ecnavi.jp,"{'a', 'n', '.', 'e', 'j', 'p', 'i', 'c', 'v'}","[ec, cn, na, av, vi, i., .j, jp]",NaN,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,53,10.0.2.15,47296,30024,hubpages.com,"{'a', 'g', '.', 'o', 'e', 'p', 'h', 's', 'b', ...","[hu, ub, bp, pa, ag, ge, es, s., .c, co, om]",NaN,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,53,10.0.2.15,47296,54860,hubpages.com,"{'a', 'g', '.', 'o', 'e', 'p', 'h', 's', 'b', ...","[hu, ub, bp, pa, ag, ge, es, s., .c, co, om]",NaN,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,53,10.0.2.15,55060,48426,discover.hubpages.com,"{'a', 'd', 'g', '.', 'o', 'e', 'u', 'h', 'p', ...","[di, is, sc, co, ov, ve, er, r., .h, hu, ub, b...",NaN,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,53,10.0.2.15,34276,43676,c1781279.ferozo.com,"{'z', '8', '1', '.', 'o', 'e', '7', '9', 'm', ...","[c1, 17, 78, 81, 12, 27, 79, 9., .f, fe, er, r...",NaN,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,53,10.0.2.15,33341,34713,c1781279.ferozo.com,"{'z', '8', '1', '.', 'o', 'e', '7', '9', 'm', ...","[c1, 17, 78, 81, 12, 27, 79, 9., .f, fe, er, r...",NaN,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,53,10.0.2.15,60738,44402,rusnod.ru,"{'d', 'n', '.', 'o', 's', 'u', 'r'}","[ru, us, sn, no, od, d., .r, ru]",NaN,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,53,10.0.2.15,50988,30583,rusnod.ru,"{'d', 'n', '.', 'o', 's', 'u', 'r'}","[ru, us, sn, no, od, d., .r, ru]",NaN,...,-1,-1,1,9,0,0,0,0,0,0


# Domaine name  3-gram

In [6]:
from nltk import ngrams
["".join(k1) for k1 in list(ngrams(df.Domaine[0],n=3))]

['ecn', 'cna', 'nav', 'avi', 'vi.', 'i.j', '.jp']

In [9]:
from nltk import ngrams
import sets
for x in df.index:
    try:
        gram3 =["".join(k1) for k1 in list(ngrams(df.Domaine[x],n=3))]
        df['3gram'][x] = gram3
    except:
        pass
    
df

C:\Users\PC1\AppData\Local\Temp/ipykernel_10444/2114781566.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['3gram'][x] = gram3
C:\Users\PC1\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,ID_packet,IP_src,UDP_sport,IP_dst,UDP_dport,DNS_id,Domaine,1gram,2gram,3gram,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,53,10.0.2.15,47533,47210,ecnavi.jp,"{'a', 'n', '.', 'e', 'j', 'p', 'i', 'c', 'v'}","[ec, cn, na, av, vi, i., .j, jp]","[ecn, cna, nav, avi, vi., i.j, .jp]",...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,53,10.0.2.15,43722,7987,ecnavi.jp,"{'a', 'n', '.', 'e', 'j', 'p', 'i', 'c', 'v'}","[ec, cn, na, av, vi, i., .j, jp]","[ecn, cna, nav, avi, vi., i.j, .jp]",...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,53,10.0.2.15,47296,30024,hubpages.com,"{'a', 'g', '.', 'o', 'e', 'p', 'h', 's', 'b', ...","[hu, ub, bp, pa, ag, ge, es, s., .c, co, om]","[hub, ubp, bpa, pag, age, ges, es., s.c, .co, ...",...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,53,10.0.2.15,47296,54860,hubpages.com,"{'a', 'g', '.', 'o', 'e', 'p', 'h', 's', 'b', ...","[hu, ub, bp, pa, ag, ge, es, s., .c, co, om]","[hub, ubp, bpa, pag, age, ges, es., s.c, .co, ...",...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,53,10.0.2.15,55060,48426,discover.hubpages.com,"{'a', 'd', 'g', '.', 'o', 'e', 'u', 'h', 'p', ...","[di, is, sc, co, ov, ve, er, r., .h, hu, ub, b...","[dis, isc, sco, cov, ove, ver, er., r.h, .hu, ...",...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,53,10.0.2.15,34276,43676,c1781279.ferozo.com,"{'z', '8', '1', '.', 'o', 'e', '7', '9', 'm', ...","[c1, 17, 78, 81, 12, 27, 79, 9., .f, fe, er, r...","[c17, 178, 781, 812, 127, 279, 79., 9.f, .fe, ...",...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,53,10.0.2.15,33341,34713,c1781279.ferozo.com,"{'z', '8', '1', '.', 'o', 'e', '7', '9', 'm', ...","[c1, 17, 78, 81, 12, 27, 79, 9., .f, fe, er, r...","[c17, 178, 781, 812, 127, 279, 79., 9.f, .fe, ...",...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,53,10.0.2.15,60738,44402,rusnod.ru,"{'d', 'n', '.', 'o', 's', 'u', 'r'}","[ru, us, sn, no, od, d., .r, ru]","[rus, usn, sno, nod, od., d.r, .ru]",...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,53,10.0.2.15,50988,30583,rusnod.ru,"{'d', 'n', '.', 'o', 's', 'u', 'r'}","[ru, us, sn, no, od, d., .r, ru]","[rus, usn, sno, nod, od., d.r, .ru]",...,-1,-1,1,9,0,0,0,0,0,0


# Domaine Name Shannon's  Entropy

In [61]:
 # Find suspicious Domaines  by calculating the Shannon entropy of the hostname.
        # Idea from: https://github.com/stricaud/urlweirdos/blob/master/src/urlw/plugins/shannon/__init__.py
        # TODO: test with unicode enabled hostnames!


Call pandas.Series(data) with data as the list of data to be analyzed to convert it into a pandas.Series. Call Series.value_counts() with the result of the previous step as Series to get a new pandas.Series containing counts of unique values in the data set. Call scipy.stats.entropy(counts) with counts as this new pandas.Series to get the Shannon entropy of the data set.

In [68]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [69]:
import scipy

In [3]:
df = loadDataset("Train_Dataset/Test_dataset_NIDS.csv")

In [28]:
import math


def entropy(string):
    "Calculates the Shannon entropy of a string"

    # get probability of chars in string
    prob = [ float(string.count(c)) / len(string) for c in dict.fromkeys(list(string)) ]

    # calculate the entropy
    entropy = - sum([ p * math.log(p) / math.log(2.0) for p in prob ])

    return entropy

In [30]:
#s= df.Domaine[0]

In [31]:
#s

'ecnavi.jp'

In [32]:
#entropy(s)

3.169925001442312

In [34]:
for x in df.index:
    hostname = df.Domaine[x]
    res =  entropy(hostname)
    df['entropy'][x] = res

df

C:\Users\PC1\AppData\Local\Temp/ipykernel_10444/1985114912.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['entropy'][x] = res


,ID_packet,IP_src,UDP_sport,IP_dst,UDP_dport,DNS_id,Domaine,1gram,2gram,3gram,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,53,10.0.2.15,47533,47210,ecnavi.jp,"{'a', 'n', '.', 'e', 'j', 'p', 'i', 'c', 'v'}","['ec', 'cn', 'na', 'av', 'vi', 'i.', '.j', 'jp']","['ecn', 'cna', 'nav', 'avi', 'vi.', 'i.j', '.jp']",...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,53,10.0.2.15,43722,7987,ecnavi.jp,"{'a', 'n', '.', 'e', 'j', 'p', 'i', 'c', 'v'}","['ec', 'cn', 'na', 'av', 'vi', 'i.', '.j', 'jp']","['ecn', 'cna', 'nav', 'avi', 'vi.', 'i.j', '.jp']",...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,53,10.0.2.15,47296,30024,hubpages.com,"{'a', 'g', '.', 'o', 'e', 'p', 'h', 's', 'b', ...","['hu', 'ub', 'bp', 'pa', 'ag', 'ge', 'es', 's....","['hub', 'ubp', 'bpa', 'pag', 'age', 'ges', 'es...",...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,53,10.0.2.15,47296,54860,hubpages.com,"{'a', 'g', '.', 'o', 'e', 'p', 'h', 's', 'b', ...","['hu', 'ub', 'bp', 'pa', 'ag', 'ge', 'es', 's....","['hub', 'ubp', 'bpa', 'pag', 'age', 'ges', 'es...",...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,53,10.0.2.15,55060,48426,discover.hubpages.com,"{'a', 'd', 'g', '.', 'o', 'e', 'u', 'h', 'p', ...","['di', 'is', 'sc', 'co', 'ov', 've', 'er', 'r....","['dis', 'isc', 'sco', 'cov', 'ove', 'ver', 'er...",...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,53,10.0.2.15,34276,43676,c1781279.ferozo.com,"{'z', '8', '1', '.', 'o', 'e', '7', '9', 'm', ...","['c1', '17', '78', '81', '12', '27', '79', '9....","['c17', '178', '781', '812', '127', '279', '79...",...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,53,10.0.2.15,33341,34713,c1781279.ferozo.com,"{'z', '8', '1', '.', 'o', 'e', '7', '9', 'm', ...","['c1', '17', '78', '81', '12', '27', '79', '9....","['c17', '178', '781', '812', '127', '279', '79...",...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,53,10.0.2.15,60738,44402,rusnod.ru,"{'d', 'n', '.', 'o', 's', 'u', 'r'}","['ru', 'us', 'sn', 'no', 'od', 'd.', '.r', 'ru']","['rus', 'usn', 'sno', 'nod', 'od.', 'd.r', '.ru']",...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,53,10.0.2.15,50988,30583,rusnod.ru,"{'d', 'n', '.', 'o', 's', 'u', 'r'}","['ru', 'us', 'sn', 'no', 'od', 'd.', '.r', 'ru']","['rus', 'usn', 'sno', 'nod', 'od.', 'd.r', '.ru']",...,-1,-1,1,9,0,0,0,0,0,0


# Domaine Name Alexa Ranking feature

Will use Alexa ranking to see the popularity of a site , this will help classify phishing domaines from bengins

In [47]:
pip install mechanize

Note: you may need to restart the kernel to use updated packages.


In [50]:
pip install alexa-siterank

  Attempting uninstall: anyio
    Found existing installation: anyio 2.2.0
    Uninstalling anyio-2.2.0:
      Successfully uninstalled anyio-2.2.0
Note: you may need to restart the kernel to use updated packages.


In [6]:
#Adding columns to the dataframe df
df['Alexa_Rank']=""
df = df[['ID_packet', 'IP_src', 'Domain', 'subdomain', 'Alexa_Rank', 'sum','obfuscate_at_sign', 'UDP_sport', 'numeric_percentage', 'IP','Registrant_Name', 'Domain_Age', 'Emails', 'Organization','Name_Server_Count', 'UDP_dport', 'DNS_id', '1gram', '2gram', '3gram','entropy', 'Country', 'tld', 'DNS_qr', 'DNS_qd', 'DNS_qdtype','type_table', 'Creation_Date_Time', 'Exp_Date_Time','whois_updatedDate', 'dot_count', 'len', 'digit_count', 'special_count','hyphen_count', 'double_slash', 'single_slash', 'at_the_rate']]

df    

,ID_packet,IP_src,Domain,subdomain,Alexa_Rank,sum,obfuscate_at_sign,UDP_sport,numeric_percentage,IP,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,ecnavi.jp,0.0,,0,0.0,53,0.000000,10.0.2.15,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,ecnavi.jp,0.0,,0,0.0,53,0.000000,10.0.2.15,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,hubpages.com,0.0,,0,0.0,53,0.000000,10.0.2.15,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,hubpages.com,0.0,,0,0.0,53,0.000000,10.0.2.15,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,discover.hubpages.com,0.0,,0,0.0,53,0.000000,10.0.2.15,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,c1781279.ferozo.com,1.0,,7,1.0,53,0.368421,10.0.2.15,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,c1781279.ferozo.com,1.0,,7,1.0,53,0.368421,10.0.2.15,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,rusnod.ru,0.0,,0,0.0,53,0.000000,10.0.2.15,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,rusnod.ru,0.0,,0,0.0,53,0.000000,10.0.2.15,...,-1,-1,1,9,0,0,0,0,0,0


In [9]:
from alexa_siterank import *

In [8]:
pip install alexa_siterank

  Using cached alexa_siterank-1.0.2-py3-none-any.whl (6.6 kB)
  Using cached httpx-0.22.0-py3-none-any.whl (84 kB)
  Using cached httpcore-0.14.7-py3-none-any.whl (68 kB)
  Using cached rfc3986-1.5.0-py2.py3-none-any.whl (31 kB)
  Using cached h11-0.12.0-py3-none-any.whl (54 kB)
  Attempting uninstall: anyio
    Found existing installation: anyio 2.2.0
    Uninstalling anyio-2.2.0:
      Successfully uninstalled anyio-2.2.0
Note: you may need to restart the kernel to use updated packages.


In [10]:
import csv

In [11]:
df.index

RangeIndex(start=0, stop=2126, step=1)

In [12]:
df.index[852::]

RangeIndex(start=852, stop=2126, step=1)

In [15]:
from alexa_siterank import OTR
import re
with open('rank.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)  
    for x in df.index:
        print(x)
        try:
            otr = OTR()
            otr.get(df.Domain[x])
            #print(df.Domaine[x])
            res = str(otr.rank)
            s1 = res.split(',', 1)[0]
            #print(res)
            #print(s1)
            s = ''.join(x for x in s1 if x.isdigit())
            print(x,df.Domain[x],s)
            df['Alexa_Rank'][x]= s 
            df.to_csv('Train_Dataset/Test_dataset_NIDS.csv',index=False)
        except:
            df['Alexa_Rank'][x]= -1.0
            continue 
        writer.writerow([s])        
df

0
1
2


C:\Users\PC1\AppData\Local\Temp/ipykernel_9600/3165882957.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Alexa_Rank'][x]= -1.0


3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278


,ID_packet,IP_src,Domain,subdomain,Alexa_Rank,sum,obfuscate_at_sign,UDP_sport,numeric_percentage,IP,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,ecnavi.jp,0.0,-1.0,0,0.0,53,0.000000,10.0.2.15,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,ecnavi.jp,0.0,-1.0,0,0.0,53,0.000000,10.0.2.15,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,hubpages.com,0.0,-1.0,0,0.0,53,0.000000,10.0.2.15,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,hubpages.com,0.0,-1.0,0,0.0,53,0.000000,10.0.2.15,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,discover.hubpages.com,0.0,-1.0,0,0.0,53,0.000000,10.0.2.15,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,c1781279.ferozo.com,1.0,-1.0,7,1.0,53,0.368421,10.0.2.15,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,c1781279.ferozo.com,1.0,-1.0,7,1.0,53,0.368421,10.0.2.15,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,rusnod.ru,0.0,-1.0,0,0.0,53,0.000000,10.0.2.15,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,rusnod.ru,0.0,-1.0,0,0.0,53,0.000000,10.0.2.15,...,-1,-1,1,9,0,0,0,0,0,0


# Save to Test datasets CSV file

In [10]:
df.to_csv('Train_Dataset/Test_dataset_NIDS.csv',index=False)

In [2]:

def loadDataset(file_name):
    df = pd.read_csv(file_name, encoding = "ISO-8859-1", on_bad_lines='skip',  dtype={"Alexa_Rank": 'float64',"entropy": 'float64',"len": 'int64', 'Country': 'string', 'IP': 'string', 'Domain' : 'string', 'Creation_Date_Time' : 'string', '1gram' : 'string','tld': 'string','3gram': 'string','2gram': 'string','label': 'float64'})
    return df

In [3]:
df = loadDataset("Train_Dataset/Test_dataset_NIDS.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2126 entries, 0 to 2125
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_packet           2126 non-null   int64  
 1   IP_src              2126 non-null   object 
 2   Domain              2126 non-null   string 
 3   subdomain           2126 non-null   float64
 4   sum                 2126 non-null   int64  
 5   obfuscate_at_sign   2126 non-null   float64
 6   UDP_sport           2126 non-null   int64  
 7   numeric_percentage  2126 non-null   float64
 8   IP                  2126 non-null   string 
 9   Registrant_Name     2126 non-null   object 
 10  Domain_Age          2126 non-null   float64
 11  Emails              2126 non-null   object 
 12  Organization        0 non-null      float64
 13  Name_Server_Count   2126 non-null   float64
 14  UDP_dport           2126 non-null   int64  
 15  DNS_id              2126 non-null   int64  
 16  1gram 

In [5]:
df.columns

Index(['ID_packet', 'IP_src', 'Domain', 'subdomain', 'sum',
       'obfuscate_at_sign', 'UDP_sport', 'numeric_percentage', 'IP',
       'Registrant_Name', 'Domain_Age', 'Emails', 'Organization',
       'Name_Server_Count', 'UDP_dport', 'DNS_id', '1gram', '2gram', '3gram',
       'entropy', 'Country', 'tld', 'DNS_qr', 'DNS_qd', 'DNS_qdtype',
       'type_table', 'Creation_Date_Time', 'Exp_Date_Time',
       'whois_updatedDate', 'dot_count', 'len', 'digit_count', 'special_count',
       'hyphen_count', 'double_slash', 'single_slash', 'at_the_rate'],
      dtype='object')

In [12]:
df

,ID_packet,IP_src,UDP_sport,IP,UDP_dport,DNS_id,Domain,Alexa_Rank,1gram,2gram,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,53,10.0.2.15,47533,47210,ecnavi.jp,5435.0,"{'a', 'n', '.', 'e', 'j', 'p', 'i', 'c', 'v'}","['ec', 'cn', 'na', 'av', 'vi', 'i.', '.j', 'jp']",...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,53,10.0.2.15,43722,7987,ecnavi.jp,5435.0,"{'a', 'n', '.', 'e', 'j', 'p', 'i', 'c', 'v'}","['ec', 'cn', 'na', 'av', 'vi', 'i.', '.j', 'jp']",...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,53,10.0.2.15,47296,30024,hubpages.com,3231.0,"{'a', 'g', '.', 'o', 'e', 'p', 'h', 's', 'b', ...","['hu', 'ub', 'bp', 'pa', 'ag', 'ge', 'es', 's....",...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,53,10.0.2.15,47296,54860,hubpages.com,3231.0,"{'a', 'g', '.', 'o', 'e', 'p', 'h', 's', 'b', ...","['hu', 'ub', 'bp', 'pa', 'ag', 'ge', 'es', 's....",...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,53,10.0.2.15,55060,48426,discover.hubpages.com,3231.0,"{'a', 'd', 'g', '.', 'o', 'e', 'u', 'h', 'p', ...","['di', 'is', 'sc', 'co', 'ov', 've', 'er', 'r....",...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,53,10.0.2.15,34276,43676,c1781279.ferozo.com,-1.0,"{'z', '8', '1', '.', 'o', 'e', '7', '9', 'm', ...","['c1', '17', '78', '81', '12', '27', '79', '9....",...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,53,10.0.2.15,33341,34713,c1781279.ferozo.com,-1.0,"{'z', '8', '1', '.', 'o', 'e', '7', '9', 'm', ...","['c1', '17', '78', '81', '12', '27', '79', '9....",...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,53,10.0.2.15,60738,44402,rusnod.ru,-1.0,"{'d', 'n', '.', 'o', 's', 'u', 'r'}","['ru', 'us', 'sn', 'no', 'od', 'd.', '.r', 'ru']",...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,53,10.0.2.15,50988,30583,rusnod.ru,-1.0,"{'d', 'n', '.', 'o', 's', 'u', 'r'}","['ru', 'us', 'sn', 'no', 'od', 'd.', '.r', 'ru']",...,-1,-1,1,9,0,0,0,0,0,0


In [15]:
#Adding columns to the dataframe df
df['Registrant_Name']=""
df['Domain_Age']=""
df['Emails']=""
df['Organization']=""
df['Name_Server_Count']=""   
df = df[['ID_packet', 'IP_src', 'UDP_sport', 'IP','Registrant_Name','Domain_Age','Emails','Organization','Name_Server_Count', 'UDP_dport', 'DNS_id','Domain','1gram','2gram','3gram','entropy','Country','tld', 'DNS_qr', 'DNS_qd', 'DNS_qdtype', 'type_table', 'Creation_Date_Time', 'Exp_Date_Time', 'whois_updatedDate', 'dot_count','len', 'digit_count', 'special_count', 'hyphen_count', 'double_slash','single_slash', 'at_the_rate']]

df    

,ID_packet,IP_src,UDP_sport,IP,Registrant_Name,Domain_Age,Emails,Organization,Name_Server_Count,UDP_dport,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,53,10.0.2.15,,,,,,47533,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,53,10.0.2.15,,,,,,43722,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,53,10.0.2.15,,,,,,47296,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,53,10.0.2.15,,,,,,47296,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,53,10.0.2.15,,,,,,55060,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,53,10.0.2.15,,,,,,34276,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,53,10.0.2.15,,,,,,33341,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,53,10.0.2.15,,,,,,60738,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,53,10.0.2.15,,,,,,50988,...,-1,-1,1,9,0,0,0,0,0,0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2126 entries, 0 to 2125
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_packet           2126 non-null   int64  
 1   IP_src              2126 non-null   object 
 2   UDP_sport           2126 non-null   int64  
 3   IP                  2126 non-null   string 
 4   Registrant_Name     2126 non-null   object 
 5   Domain_Age          2126 non-null   object 
 6   Emails              2126 non-null   object 
 7   Organization        2126 non-null   object 
 8   Name_Server_Count   2126 non-null   object 
 9   UDP_dport           2126 non-null   int64  
 10  DNS_id              2126 non-null   int64  
 11  Domain              2126 non-null   string 
 12  1gram               2126 non-null   string 
 13  2gram               2126 non-null   string 
 14  3gram               2126 non-null   string 
 15  entropy             2126 non-null   float64
 16  Countr

In [19]:
import whois
import datetime
import  csv

In [38]:
import requests
from bs4 import BeautifulSoup
import re

In [42]:
pip uninstall whois 

^C
Note: you may need to restart the kernel to use updated packages.


In [43]:
pip install python-whois

Note: you may need to restart the kernel to use updated packages.


# Extract Registar Name and Domain Age

In [49]:
#with open('age.csv', 'w') as csvfile:
  #  writer = csv.writer(csvfile)  

#url = "http://www.webconfs.com/domain-age.php"

for x in df.index:
    try:
        #whois query 
        #print(df.Domain[x])
        response= whois.whois(df.Domain[x])
    #print("The Domain name's:", response.domain_name)
        print("Domain Created on: ", response.creation_date.strftime('%d %B %Y'))
        print("Domain Registered By: ", response.registrar)
        S= response.registrar
        #age = (datetime.datetime.today() - response.creation_date).days 
        print("The Age of Domain is: ", age,"days")
    #print(S,age)
        df['Registrant_Name'][x]= S 
        #df['Domain_Age'][x]= age
    except:
        df['Registrant_Name'][x]= ' '
        #df['Domain_Age'][x]= 0.0
        continue 
       # writer.writerow([s])   
        
df       

C:\Users\PC1\AppData\Local\Temp/ipykernel_4488/1837112228.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Registrant_Name'][x]= ' '
C:\Users\PC1\AppData\Local\Temp/ipykernel_4488/1837112228.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Domain_Age'][x]= 0.0


,ID_packet,IP_src,UDP_sport,IP,Registrant_Name,Domain_Age,Emails,Organization,Name_Server_Count,UDP_dport,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,53,10.0.2.15,,0.0,,,,47533,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,53,10.0.2.15,,0.0,,,,43722,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,53,10.0.2.15,,0.0,,,,47296,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,53,10.0.2.15,,0.0,,,,47296,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,53,10.0.2.15,,0.0,,,,55060,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,53,10.0.2.15,,0.0,,,,34276,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,53,10.0.2.15,,0.0,,,,33341,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,53,10.0.2.15,,0.0,,,,60738,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,53,10.0.2.15,,0.0,,,,50988,...,-1,-1,1,9,0,0,0,0,0,0


# Domain name Age extractor in Days

In [6]:
import subprocess
from datetime  import *
import dateutil.parser as p

In [ ]:
#url1 = "itb.ie"
#url1 = "night-fever.it"
#list_of_Domain = df['Domain'].to_list()

 """
    Uses whois and grep to return number of days a domain has been alive based
    on the created/registered date.
    Parameters:
    argument1 (String): domain to check
    Returns:
    int: -1 No match from whois - domain does not exist
         -2 whois didn't return what we expected - no reg date
         -3 there was a problem parsing a valid date from the whois data
    Note1: This function is dependant on whois and grep being available form the command line
    Note2: This function is also dependant on the dateutil library: pip install py-dateutil
   """
def getDaysAlive(url:str):
    # Use grep to strip out the part of the output that we need
    grep_filter = " | grep -E \"Registration Date|Registered on|Creation Date|created:|Created:|Registration Time:|No match for domain\""

    # Call the whois and pipe the output to grep
    whois_data = subprocess.Popen("whois " + url + grep_filter, shell=True, stdout=subprocess.PIPE).stdout.read()

    # whois could not find a match for the domain - doesn't exist
    if ("No match for domain" in whois_data):
        return -1

    # Split the output based on carriage returns (each line of output from grep)
    whois_data = whois_data.strip().split('\n')

    # Only one date entry found - should be two ?? (Server followed by creation date of domain)
    if (len(whois_data) == 1):
        return -2

    # Try to parse a datetime object out of the string
    # NOTE: we are assuming that the last entry in the list returned from whois and grep
    # will be the actual registration/creation data of the domain in question: whois_data[-1]
    try:
        # Fingers crossed we get a valid date out of the string
        reg_date =  p.parse(whois_data[-1].lower(), fuzzy=True)
    except:
        return -3
    # Get datetime stamp based on NOW!
    today = datetime.today()
    # Timezone and no timezones can cause problems when comparing
    # Strip timezone info from each datetime object (not a very good idea - fudge!!)
    today = today.replace(tzinfo=None)
    reg_date = reg_date.replace(tzinfo=None)
    # Return the days alive (Diff between dates)
    return today - reg_date



for x in df.index:
# Test the function with a list of domain names in the test dataset...........
        # days_alive = getDaysAlive(domain)
    print(domain)
    # getDaysAlive(domain)
    age = getDaysAlive(df.Domain[x])
    df['Domain_Age'][x]= age 

df   

# Email Extractor in Python  : Scrap Email address from Doamin Names

In [52]:
pip install requests-html

  Using cached cssselect-1.1.0-py2.py3-none-any.whl (16 kB)

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1271 sha256=ee6c64ee272b18b90f0293d8989fa61f0fc084421f179075a9f3a70a3321294f
  Stored in directory: c:\users\pc1\appdata\local\pip\cache\wheels\73\2b\cb\099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=12fa77579a047e0cfc52b6ea0870c55677cf4e097726470f711ad027f86b2c02
  Stored in directory: c:\users\pc1\appdata\local\pip\cache\wheels\ae\e7\76\7dd44644d065268ab0e1b4fa2e802fa4bb0157717b7d6c6d92
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=0d50d35cd4ed2bf8ed039a93cdc0c72ae53fe37b5f1d3e174d9d344ade4105f3
  Stored in directory: c:\users\pc1\appdata\local\pip\cache\wheels\d6\9c\58\ee3ba36897e890f3ad81e9b730791a153fce20caa4a8a474df
Successfully built bs4 fake-useragent parse


In [53]:
import re
from requests_html import HTMLSession

We need re module here because we will be extracting emails from HTML content using regular expressions, if you're not sure what a regular expression is, it is basically a sequence of characters that define a search pattern.

I've grabbed the most used and accurate regular expression for email addresses from a stackoverflow tred

url string is the URL/domaine name we want to grab email addresses from, I'm using a website that generates random email addresses (which loads them using Javascript).

Let's initiate the HTML session, which is a consumable session for cookie persistence and connection pooling:

In [64]:
for x in df.index:
    try:   
    #print(df.Domain[x])
        url= "http://" + df.Domain[x]
        print(url)
        EMAIL_REGEX = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""
        # initiate an HTTP session
        session = HTMLSession()
        # get the HTTP Response
        r = session.get(url)
        for re_match in re.finditer(EMAIL_REGEX, r.html.raw_html.decode()):
            print(re_match.group())
            S = re_match.group()
            df['Emails'][x]= S 
    except:
        df['Emails'][x]= ' '
        continue 
       # writer.writerow([s])   
                
df       

http://ecnavi.jp
http://ecnavi.jp
http://hubpages.com
http://hubpages.com
http://discover.hubpages.com
http://discover.hubpages.com
http://hubpages.com
http://extratorrent.cc
http://extratorrent.cc
http://icicibank.com
nri@icicibank.com
http://icicibank.com


C:\Users\PC1\AppData\Local\Temp/ipykernel_4488/2936297315.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Emails'][x]= S


nri@icicibank.com
http://www.icicibank.com
nri@icicibank.com
http://www.icicibank.com
nri@icicibank.com
http://icicibank.com
nri@icicibank.com
http://nypost.com
//nypost.com/wp-content/uploads/sites/2/2021/12/banner@2x.png
http://nypost.com
//nypost.com/wp-content/uploads/sites/2/2021/12/banner@2x.png
http://nypost.com
//nypost.com/wp-content/uploads/sites/2/2021/12/banner@2x.png
http://kienthuc.net.vn
baotrithuccuocsong@gmail.com
baotrithuccuocsong@gmail.com
tkts@kienthuc.net.vn
tkts@kienthuc.net.vn
http://kienthuc.net.vn
baotrithuccuocsong@gmail.com
baotrithuccuocsong@gmail.com
tkts@kienthuc.net.vn
tkts@kienthuc.net.vn
http://kienthuc.net.vn
baotrithuccuocsong@gmail.com
baotrithuccuocsong@gmail.com
tkts@kienthuc.net.vn
tkts@kienthuc.net.vn
http://thenextweb.com
justinribeiro/lite-youtube@1.3.1
//next.tnwcdn.com/assets/img/conference-ad-new-900x750@3x.jpg
//next.tnwcdn.com/assets/img/conference-ad-new-300x250@1x.jpg
//next.tnwcdn.com/assets/img/conference-ad-new-300x250@2x.jpg
//next.

C:\Users\PC1\AppData\Local\Temp/ipykernel_4488/2936297315.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Emails'][x]= ' '


http://sfglobe.com
http://mic.com
http://mic.com
http://www.mic.com
http://www.mic.com
http://mic.com
http://venturebeat.com
partners@venturebeat.com
http://venturebeat.com
partners@venturebeat.com
http://venturebeat.com
partners@venturebeat.com
http://allegro.pl
http://allegro.pl
http://metro.co.uk
http://metro.co.uk
http://metro.co.uk
http://atwiki.jp
http://atwiki.jp
http://atwiki.jp
http://nguyentandung.org
http://nguyentandung.org
http://nguyentandung.org
http://spankbang.com
mages/logo_v5@2x.png
mages/logo_desktop_v4@2x.png
http://spankbang.com
mages/logo_v5@2x.png
mages/logo_desktop_v4@2x.png
http://spankbang.com
mages/logo_v5@2x.png
mages/logo_desktop_v4@2x.png
http://torcache.net
http://torcache.net
http://depositphotos.com
http://depositphotos.com
http://depositphotos.com
http://serverfault.com
mg/apple-touch-icon@2.png
http://serverfault.com
mg/apple-touch-icon@2.png
http://serverfault.com
mg/apple-touch-icon@2.png
http://kakaku.com
http://kakaku.com
http://kakaku.com
http:/

,ID_packet,IP_src,UDP_sport,IP,Registrant_Name,Domain_Age,Emails,Organization,Name_Server_Count,UDP_dport,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,53,10.0.2.15,,0.0,,,,47533,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,53,10.0.2.15,,0.0,,,,43722,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,53,10.0.2.15,,0.0,,,,47296,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,53,10.0.2.15,,0.0,,,,47296,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,53,10.0.2.15,,0.0,,,,55060,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,53,10.0.2.15,,0.0,,,,34276,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,53,10.0.2.15,,0.0,,,,33341,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,53,10.0.2.15,,0.0,,,,60738,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,53,10.0.2.15,,0.0,regionnod@yandex.ru,,,50988,...,-1,-1,1,9,0,0,0,0,0,0


If  the website you're grabbing email addresses from uses JavaScript to load most of the data, then you need to execute the below line of code:

In [ ]:
# for JAVA-Script driven websites
r.html.render()

In [122]:
df.to_csv('Train_Dataset/Test_dataset_NIDS.csv',index=False)

In [66]:
df

,ID_packet,IP_src,UDP_sport,IP,Registrant_Name,Domain_Age,Emails,Organization,Name_Server_Count,UDP_dport,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,53,10.0.2.15,,0.0,,,,47533,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,53,10.0.2.15,,0.0,,,,43722,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,53,10.0.2.15,,0.0,,,,47296,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,53,10.0.2.15,,0.0,,,,47296,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,53,10.0.2.15,,0.0,,,,55060,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,53,10.0.2.15,,0.0,,,,34276,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,53,10.0.2.15,,0.0,,,,33341,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,53,10.0.2.15,,0.0,,,,60738,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,53,10.0.2.15,,0.0,regionnod@yandex.ru,,,50988,...,-1,-1,1,9,0,0,0,0,0,0


# Find Nameservers count of domain name

In [7]:
pip install dnspython

Note: you may need to restart the kernel to use updated packages.


In [11]:
import dns.resolver

In [16]:
for x in df.index:
    try:
        nameservers = dns.resolver.query(df.Domain[x],'NS')
        for data in nameservers:
            print(data)
            df['Name_Server_Count'][x] = len(nameservers)
            print(len(nameservers))
    except:
        #df['Emails'][x]= ' '
        continue 
       # writer.writerow([s])   
                
df          

C:\Users\PC1\AppData\Local\Temp/ipykernel_18828/1547494076.py:3: DeprecationWarning: please use dns.resolver.resolve() instead
  nameservers = dns.resolver.query(df.Domain[x],'NS')
C:\Users\PC1\AppData\Local\Temp/ipykernel_18828/1547494076.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Name_Server_Count'][x] = len(nameservers)


ns-751.awsdns-29.net.
4
ns-1097.awsdns-09.org.
4
ns-469.awsdns-58.com.
4
ns-1830.awsdns-36.co.uk.
4
ns-1097.awsdns-09.org.
4
ns-751.awsdns-29.net.
4
ns-1830.awsdns-36.co.uk.
4
ns-469.awsdns-58.com.
4
ns-1345.awsdns-40.org.
4
ns-577.awsdns-08.net.
4
ns-1714.awsdns-22.co.uk.
4
ns-413.awsdns-51.com.
4
ns-413.awsdns-51.com.
4
ns-1345.awsdns-40.org.
4
ns-577.awsdns-08.net.
4
ns-1714.awsdns-22.co.uk.
4
ns-413.awsdns-51.com.
4
ns-1714.awsdns-22.co.uk.
4
ns-1345.awsdns-40.org.
4
ns-577.awsdns-08.net.
4
zara.ns.cloudflare.com.
2
hugh.ns.cloudflare.com.
2
hugh.ns.cloudflare.com.
2
zara.ns.cloudflare.com.
2
a20-67.akam.net.
9
ns3.icicibank.com.
9
a14-65.akam.net.
9
ns2.icicibank.com.
9
a7-65.akam.net.
9
ns1.icicibank.com.
9
a3-66.akam.net.
9
a1-79.akam.net.
9
a12-65.akam.net.
9
a1-79.akam.net.
9
a7-65.akam.net.
9
a14-65.akam.net.
9
a20-67.akam.net.
9
a3-66.akam.net.
9
ns2.icicibank.com.
9
a12-65.akam.net.
9
ns1.icicibank.com.
9
ns3.icicibank.com.
9
a14-65.akam.net.
9
a20-67.akam.net.
9
ns1.icicib

,ID_packet,IP_src,Domain,subdomain,UDP_sport,numeric_percentage,IP,Registrant_Name,Domain_Age,Emails,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,ecnavi.jp,0.0,53,0.000000,10.0.2.15,,0.0,,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,ecnavi.jp,0.0,53,0.000000,10.0.2.15,,0.0,,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,hubpages.com,0.0,53,0.000000,10.0.2.15,,0.0,,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,hubpages.com,0.0,53,0.000000,10.0.2.15,,0.0,,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,discover.hubpages.com,1.0,53,0.000000,10.0.2.15,,0.0,,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,c1781279.ferozo.com,1.0,53,0.368421,10.0.2.15,,0.0,,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,c1781279.ferozo.com,1.0,53,0.368421,10.0.2.15,,0.0,,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,rusnod.ru,0.0,53,0.000000,10.0.2.15,,0.0,,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,rusnod.ru,0.0,53,0.000000,10.0.2.15,,0.0,regionnod@yandex.ru,...,-1,-1,1,9,0,0,0,0,0,0


# Numeric percentages of Number digit in Domain name

In [99]:
#Adding columns to the dataframe df
df['numeric_percentage']=" "
   
df = df[['ID_packet', 'IP_src','Domain', 'UDP_sport','numeric_percentage', 'IP','Registrant_Name','Domain_Age','Emails','Organization','Name_Server_Count', 'UDP_dport', 'DNS_id','1gram','2gram','3gram','entropy','Country','tld', 'DNS_qr', 'DNS_qd', 'DNS_qdtype', 'type_table', 'Creation_Date_Time', 'Exp_Date_Time', 'whois_updatedDate', 'dot_count','len', 'digit_count', 'special_count', 'hyphen_count', 'double_slash','single_slash', 'at_the_rate']]

df    

C:\Users\PC1\AppData\Local\Temp/ipykernel_4488/3049554657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['numeric_percentage']=" "


,ID_packet,IP_src,Domain,UDP_sport,numeric_percentage,IP,Registrant_Name,Domain_Age,Emails,Organization,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,ecnavi.jp,53,,10.0.2.15,,0.0,,,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,ecnavi.jp,53,,10.0.2.15,,0.0,,,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,hubpages.com,53,,10.0.2.15,,0.0,,,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,hubpages.com,53,,10.0.2.15,,0.0,,,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,discover.hubpages.com,53,,10.0.2.15,,0.0,,,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,c1781279.ferozo.com,53,,10.0.2.15,,0.0,,,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,c1781279.ferozo.com,53,,10.0.2.15,,0.0,,,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,rusnod.ru,53,,10.0.2.15,,0.0,,,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,rusnod.ru,53,,10.0.2.15,,0.0,regionnod@yandex.ru,,...,-1,-1,1,9,0,0,0,0,0,0


In [111]:
df['Domain']

0                   ecnavi.jp
1                   ecnavi.jp
2                hubpages.com
3                hubpages.com
4       discover.hubpages.com
                ...          
2121      c1781279.ferozo.com
2122      c1781279.ferozo.com
2123                rusnod.ru
2124                rusnod.ru
2125                rusnod.ru
Name: Domain, Length: 2126, dtype: string

In [117]:
for x in df.index:
    df['numeric_percentage'][x] = float(len(''.join(re.findall('\d',df.Domain[x]))))/float(len(df.Domain[x]))
    
df

C:\Users\PC1\AppData\Local\Temp/ipykernel_4488/3068978542.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['numeric_percentage'][x] = float(len(''.join(re.findall('\d',df.Domain[x]))))/float(len(df.Domain[x]))


,ID_packet,IP_src,Domain,UDP_sport,numeric_percentage,IP,Registrant_Name,Domain_Age,Emails,Organization,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,ecnavi.jp,53,0.0,10.0.2.15,,0.0,,,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,ecnavi.jp,53,0.0,10.0.2.15,,0.0,,,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,hubpages.com,53,0.0,10.0.2.15,,0.0,,,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,hubpages.com,53,0.0,10.0.2.15,,0.0,,,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,discover.hubpages.com,53,0.0,10.0.2.15,,0.0,,,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,c1781279.ferozo.com,53,0.368421,10.0.2.15,,0.0,,,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,c1781279.ferozo.com,53,0.368421,10.0.2.15,,0.0,,,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,rusnod.ru,53,0.0,10.0.2.15,,0.0,,,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,rusnod.ru,53,0.0,10.0.2.15,,0.0,regionnod@yandex.ru,,...,-1,-1,1,9,0,0,0,0,0,0


# Subdomain extraction 

In [10]:
#Adding columns to the dataframe df
df['subdomain']=" "
   
df = df[['ID_packet', 'IP_src','Domain','subdomain', 'UDP_sport','numeric_percentage', 'IP','Registrant_Name','Domain_Age','Emails','Organization','Name_Server_Count', 'UDP_dport', 'DNS_id','1gram','2gram','3gram','entropy','Country','tld', 'DNS_qr', 'DNS_qd', 'DNS_qdtype', 'type_table', 'Creation_Date_Time', 'Exp_Date_Time', 'whois_updatedDate', 'dot_count','len', 'digit_count', 'special_count', 'hyphen_count', 'double_slash','single_slash', 'at_the_rate']]


In [13]:
import re
def extract_domain(domain):
    domain = re.sub('http(s)?://|(\:|/)(.*)|','', domain)
    matches = re.findall("([a-z0-9][a-z0-9\-]{1,63}\.[a-z\.]{2,6})$", domain)
    if matches:
        return matches[0]
    else:
        return domain
def extract_subdomains(domain):
    subdomains = domain = re.sub('http(s)?://|(\:|/)(.*)|','', domain)
    domain = extract_domain(subdomains)
    subdomains = re.sub('\.?'+domain,'', subdomains)
    return subdomains

In [26]:
for x in df.index:
    res= extract_subdomains(df.Domain[x])
    #print(res)
    if res =='':
        df['subdomain'][x] = 0.0
    else:
        df['subdomain'][x] = 1.0
    
df

C:\Users\PC1\AppData\Local\Temp/ipykernel_6828/2484516774.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subdomain'][x] = 0.0
C:\Users\PC1\AppData\Local\Temp/ipykernel_6828/2484516774.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subdomain'][x] = 1.0


,ID_packet,IP_src,Domain,subdomain,UDP_sport,numeric_percentage,IP,Registrant_Name,Domain_Age,Emails,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,ecnavi.jp,0.0,53,0.000000,10.0.2.15,,0.0,,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,ecnavi.jp,0.0,53,0.000000,10.0.2.15,,0.0,,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,hubpages.com,0.0,53,0.000000,10.0.2.15,,0.0,,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,hubpages.com,0.0,53,0.000000,10.0.2.15,,0.0,,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,discover.hubpages.com,1.0,53,0.000000,10.0.2.15,,0.0,,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,c1781279.ferozo.com,1.0,53,0.368421,10.0.2.15,,0.0,,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,c1781279.ferozo.com,1.0,53,0.368421,10.0.2.15,,0.0,,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,rusnod.ru,0.0,53,0.000000,10.0.2.15,,0.0,,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,rusnod.ru,0.0,53,0.000000,10.0.2.15,,0.0,regionnod@yandex.ru,...,-1,-1,1,9,0,0,0,0,0,0


In [17]:
df.to_csv('Train_Dataset/Test_dataset_NIDS.csv',index=False)

# Obfuscation max values 

In [22]:
#Adding columns to the dataframe df
df['obfuscate_at_sign']=" "
df['sum']=" "
   
df = df[['ID_packet', 'IP_src','Domain','subdomain','sum','obfuscate_at_sign', 'UDP_sport','numeric_percentage', 'IP','Registrant_Name','Domain_Age','Emails','Organization','Name_Server_Count', 'UDP_dport', 'DNS_id','1gram','2gram','3gram','entropy','Country','tld', 'DNS_qr', 'DNS_qd', 'DNS_qdtype', 'type_table', 'Creation_Date_Time', 'Exp_Date_Time', 'whois_updatedDate', 'dot_count','len', 'digit_count', 'special_count', 'hyphen_count', 'double_slash','single_slash', 'at_the_rate']]
df

,ID_packet,IP_src,Domain,subdomain,sum,obfuscate_at_sign,UDP_sport,numeric_percentage,IP,Registrant_Name,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,ecnavi.jp,0.0,,,53,0.000000,10.0.2.15,,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,ecnavi.jp,0.0,,,53,0.000000,10.0.2.15,,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,hubpages.com,0.0,,,53,0.000000,10.0.2.15,,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,hubpages.com,0.0,,,53,0.000000,10.0.2.15,,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,discover.hubpages.com,1.0,,,53,0.000000,10.0.2.15,,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,c1781279.ferozo.com,1.0,,,53,0.368421,10.0.2.15,,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,c1781279.ferozo.com,1.0,,,53,0.368421,10.0.2.15,,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,rusnod.ru,0.0,,,53,0.000000,10.0.2.15,,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,rusnod.ru,0.0,,,53,0.000000,10.0.2.15,,...,-1,-1,1,9,0,0,0,0,0,0


In [28]:
dr = df[['digit_count', 'special_count', 'hyphen_count', 'double_slash', 'single_slash', 'at_the_rate']].copy()

In [29]:
dr

,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
2121,7,0,0,0,0,0
2122,7,0,0,0,0,0
2123,0,0,0,0,0,0
2124,0,0,0,0,0,0


In [30]:
column_list = list(dr)

In [31]:
print(column_list)

['digit_count', 'special_count', 'hyphen_count', 'double_slash', 'single_slash', 'at_the_rate']


In [32]:
df["sum"] = dr[column_list].sum(axis=1)

In [33]:
df

,ID_packet,IP_src,Domain,subdomain,sum,obfuscate_at_sign,UDP_sport,numeric_percentage,IP,Registrant_Name,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,ecnavi.jp,0.0,0,,53,0.000000,10.0.2.15,,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,ecnavi.jp,0.0,0,,53,0.000000,10.0.2.15,,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,hubpages.com,0.0,0,,53,0.000000,10.0.2.15,,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,hubpages.com,0.0,0,,53,0.000000,10.0.2.15,,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,discover.hubpages.com,1.0,0,,53,0.000000,10.0.2.15,,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,c1781279.ferozo.com,1.0,7,,53,0.368421,10.0.2.15,,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,c1781279.ferozo.com,1.0,7,,53,0.368421,10.0.2.15,,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,rusnod.ru,0.0,0,,53,0.000000,10.0.2.15,,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,rusnod.ru,0.0,0,,53,0.000000,10.0.2.15,,...,-1,-1,1,9,0,0,0,0,0,0


In [37]:
for x in df.index:
    if df['sum'][x]>0:
        df['obfuscate_at_sign'][x] = 1.0
    else:
        df['obfuscate_at_sign'][x] = 0.0
    
df

C:\Users\PC1\AppData\Local\Temp/ipykernel_18828/3257886416.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['obfuscate_at_sign'][x] = 0.0
C:\Users\PC1\AppData\Local\Temp/ipykernel_18828/3257886416.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['obfuscate_at_sign'][x] = 1.0


,ID_packet,IP_src,Domain,subdomain,sum,obfuscate_at_sign,UDP_sport,numeric_percentage,IP,Registrant_Name,...,Exp_Date_Time,whois_updatedDate,dot_count,len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate
0,2,192.168.8.1,ecnavi.jp,0.0,0,0.0,53,0.000000,10.0.2.15,,...,-1,-1,1,9,0,0,0,0,0,0
1,78,192.168.8.1,ecnavi.jp,0.0,0,0.0,53,0.000000,10.0.2.15,,...,-1,-1,1,9,0,0,0,0,0,0
2,80,192.168.8.1,hubpages.com,0.0,0,0.0,53,0.000000,10.0.2.15,,...,-1,-1,1,12,0,0,0,0,0,0
3,81,192.168.8.1,hubpages.com,0.0,0,0.0,53,0.000000,10.0.2.15,,...,-1,-1,1,12,0,0,0,0,0,0
4,98,192.168.8.1,discover.hubpages.com,0.0,0,0.0,53,0.000000,10.0.2.15,,...,-1,-1,2,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,23221,192.168.8.1,c1781279.ferozo.com,1.0,7,1.0,53,0.368421,10.0.2.15,,...,-1,-1,2,19,7,0,0,0,0,0
2122,23222,192.168.8.1,c1781279.ferozo.com,1.0,7,1.0,53,0.368421,10.0.2.15,,...,-1,-1,2,19,7,0,0,0,0,0
2123,23223,192.168.8.1,rusnod.ru,0.0,0,0.0,53,0.000000,10.0.2.15,,...,-1,-1,1,9,0,0,0,0,0,0
2124,23224,192.168.8.1,rusnod.ru,0.0,0,0.0,53,0.000000,10.0.2.15,,...,-1,-1,1,9,0,0,0,0,0,0


In [39]:
df.to_csv('Train_Dataset/Test_dataset_NIDS.csv',index=False)